In [3]:
using BenchmarkTools, LinearAlgebra, BandedMatrices, Polynomials, PyPlot, LaTeXStrings, CSV, DataFrames, Dates
include("backSub.jl")
include("invBiU.jl")

BLAS.get_num_threads()


# BLAS.set_num_threads(8)
# BLAS.get_num_threads()


8

In [1]:
using BenchmarkTools, LinearAlgebra, BandedMatrices, PyPlot, LaTeXStrings, CSV, DataFrames, Dates

include("genTestMat.jl")
include("backsub.jl")
include("invBiU.jl")
include("invBiUExp.jl")


function timeAvg4func(func, testMat, rept, nThreads)
    BLAS.set_num_threads(nThreads)
    time = 0.0
    for _ in 1:rept
        time += @elapsed func(testMat)
    end
    time / rept;
end


function timesData4funcs()
    dims = [2^i for i = 4:6]
    # dims = [Int64(floor(2^(i/2))) for i in 4:22]
    # dims = [100*2^i for i in 0:7] # for bidiag only
    bw = 1
    # typeM = Matrix
    typeM = Bidiagonal
    typeElmt = Float64
    isUpper = true
    rept = 8
    rept = 1
    nThreads = 1
    funcs = [inv, invBidiagU, invBiUexp]
    saveData = true
    saveFig = true
    saveData = false
    saveFig = false


    timesData = zeros(length(dims), length(funcs))

    BLAS.set_num_threads(nThreads)
    for (j, func) in enumerate(funcs)
        for (i, n) in enumerate(dims)
            testMat = generateTestTriangular(n, bw, typeM, typeElmt, isUpper)
            timesData[i, j] = timeAvg4func(func, testMat, rept, nThreads)
        end
    end
    BLAS.set_num_threads(8)

    if saveData
        df_data = DataFrame(timesData, :auto)
        rename!(df_data, string.(funcs))
        df_dims = DataFrame(size = dims)
        df = hcat(df_dims, df_data)
        time = string(now())
        CSV.write("./data/efficiency/$time$typeM$typeElmt.csv", df)
        # CSV.write("./data/accuracy/$time$typeM$Float64.csv", df)
    end

    linestylelist = ["--", "-.", ":", "-", ]

    figure()
    for (j, func) in enumerate(funcs)
        # loglog(dims, timesData[:, j], color=colorlist[j], linewidth=1.0, linestyle=linestylelist[j], base=2, label=string(func))
        loglog(dims, timesData[:, j], linewidth=1.0, linestyle=linestylelist[j], base=2, label=string(func))
    end
    xlabel(L"Dimension"), ylabel("Wall time (s)"), grid(), legend()
    title(L"Time usage for computing $U^{-1} \in \mathbb{R}^{N \times N}$")
    show()
    if saveFig
        savefig("./figure/efficiency/$time$typeM$typeElmt.png", dpi=150)
        savefig("./figure/efficiency/$time$typeM$typeElmt.eps", format="eps")
    end
    timesData;
end


timesData4funcs()


sys:1: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


3×3 Matrix{Float64}:
 3.167e-6  9.209e-6   1.8833e-5
 4.542e-6  6.125e-6   1.4209e-5
 2.35e-5   2.2959e-5  4.625e-5

In [16]:
# '-', '--', '-.', ':', 'None', ' ', '', 'solid', 'dashed', 'dashdot', 'dotted'"


ls = ["-", "--", "-.", ":", "None", " ", "", "solid", "dashed", "dashdot", "dotted"]
n = length(ls)
figure()
x = [i for i = 1:n]
for i = 1:n
    plot(x, i.+x, linewidth=1.0, linestyle=ls[i], label=ls[i])
#     # plot(dims, timedata[i, :], color=colorlist[i], linewidth=1.0, linestyle=linestylelist[i], label=string(func))
end
legend()


PyObject <matplotlib.legend.Legend object at 0x16b766a60>

In [11]:
[Int64(floor(2^(i/2))) for i in 4:20]
[100*2^i for i in 0:8] # 25600 max on PC

9-element Vector{Int64}:
   100
   200
   400
   800
  1600
  3200
  6400
 12800
 25600

In [20]:
timesData = [0.0001743625 0.00025555 0.00030562499999999996;
0.0006766125 0.0007050375 0.0015038875000000001;
0.0017297875 0.03381915 0.0069719125;
0.018351212500000002 0.0137397625 0.0641487375;
0.023570050000000002 0.0491157375 0.10347664999999999;
0.08602725 0.18803486250000004 0.42850340000000003;
0.3321045625 0.7291467625000001 1.6423025625;
1.3163562625 4.641997125 7.889353975;
5.2110446625 18.790740837500003 29.9962767875]
# 8 repet 
   100
   200
   400
   800
  1600
  3200
  6400
 12800
 25600

timesData = [0.00021046874999999996 0.00018445625000000003 0.0004663875;
0.0005307624999999999 0.0007891562499999999 0.0016212687499999998;
0.00512938125 0.002715525 0.0066459250000000004;
0.044786206249999995 0.026170637499999996 0.037271543750000004;
0.0314148625 0.047623925 0.0882020375;
0.08859593125000001 0.19449356875 0.3707189875;
0.32561938125 0.8267060000000002 1.4847826874999999;
1.251984325 3.931247725 7.071467856250001;
5.14099573125 14.507024368749999 29.656756274999996]
# 25600 x 16 avg on PC
# 51200 x 1 avg on PC

# 16 repet 
100
200
400
800
1600
3200
6400
12800
25600


dims = [100*2^i for i in 0:8] # for bidiag only
bw = 1
# typeM = Matrix
typeM = Bidiagonal
typeElmt = Float64
isUpper = true
rept = 8
rept = 1
nThreads = 1
funcs = [inv, invBidiagU, invBiUexp]
saveData = true
saveFig = true
saveData = false
saveFig = false


if saveData
    df_data = DataFrame(timesData, :auto)
    rename!(df_data, string.(funcs))
    df_dims = DataFrame(size = dims)
    df = hcat(df_dims, df_data)
    time = string(now())
    # CSV.write("./data/efficiency/$time$typeM$Float64.csv", df)
    CSV.write("/Users/zhiwei_zhou/ic/project/msc_project/formal/invU/data/efficiency/$time$typeM$Float64.csv", df)
    # CSV.write("./data/accuracy/$time$typeM$Float64.csv", df)
end

linestylelist = ["--", "-.", ":", "-", ]
figure()
for (j, func) in enumerate(funcs)
    # loglog(dims, timesData[:, j], color=colorlist[j], linewidth=1.0, linestyle=linestylelist[j], base=2, label=string(func))
    loglog(dims, timesData[:, j], linewidth=1.0, linestyle=linestylelist[j], base=2, label=string(func))
end
xlabel(L"Dimension of $U$"), ylabel("Wall time (s)")
# ylabel("Wall time (s)")
grid()
legend()
title(L"Time usage for computing $U^{-1}$")
show()
if saveFig
    # savefig("./figure/efficiency/$time$typeM$typeElmt.png", dpi=150)
    # savefig("./figure/efficiency/$time$typeM$typeElmt.eps", format="eps")

    savefig("/Users/zhiwei_zhou/ic/project/msc_project/formal/invU/figure/efficiency/$time$typeM$typeElmt.png", dpi=150)
    savefig("/Users/zhiwei_zhou/ic/project/msc_project/formal/invU/figure/efficiency/$time$typeM$typeElmt.eps", format="eps")


end
timesData;

linestylelist = ["--", "-.", ":", "-", ]
dims = [100*2^i for i in 0:8] # for bidiag only
funcs = [inv, invBidiagU, invBiUexp]
figure()

# timesData = rand()
for (j, func) in enumerate(funcs)
    loglog(dims, timesData[:, j], linewidth=1.0, linestyle=linestylelist[j], base=2, label=string(func))
end
xlabel(L"Dimension"), ylabel("Wall time (s)"), grid(), legend()
title(L"Time usage for computing $U^{-1} \in \mathbb{R}^{N \times N}$")
# show()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


PyObject Text(0.5, 1.0, 'Time usage for computing $U^{-1} \\in \\mathbb{R}^{N \\times N}$')

In [31]:
m, n = 3, 3
A = zeros(m, 2*n)

for i in 1:m
    for j in 1:n
        a = j*ones(2)
        A[i,2(j-1)+1:2j] = a
    end
end
A

3×6 Matrix{Float64}:
 1.0  1.0  2.0  2.0  3.0  3.0
 1.0  1.0  2.0  2.0  3.0  3.0
 1.0  1.0  2.0  2.0  3.0  3.0

In [2]:
include("symTriSysSolver.jl")
include("symTriSysSolverexp.jl")
include("genTestMat.jl")
include("invSymTriSS.jl")
include("invSymTriSSexp.jl")

invSymT2SSexp_U (generic function with 2 methods)

In [34]:
function CholeskyUtU(U, b)
    y = U' \ b
    x = U \ y
    x;
end


function symTriSS(U, b)
    xhat, yhat = invSymT2SS_U(U, false)
    x = solver4seqTxb(xhat, yhat, b)
    x;
end


function symTriSSexp(U, b)
    (sxh, xhE), (syh, yhE) = invSymT2SSexp_U(U, false)
    x = solver4seqTxbexp((sxh, xhE), (syh, yhE), b)
    x;
end


function seqTxb(func, T, u0, seqlength)
    C = cholesky(Matrix(T))
    U = Bidiagonal(C.U)

    n = length(u0)
    u = zeros(n, seqlength)
    u[:, 1] = u0
    for j in 2:seqlength
        u[:, j] = func(U, u[:, j-1])
    end
    u;
end






seqTxb (generic function with 1 method)

In [50]:
n = 1000
T = generateTestTridiagonal(n, Matrix, Float64, true)
u0 = rand(n)

seqlength = n

1000

In [51]:
u_UtU = seqTxb(CholeskyUtU, T, u0, seqlength)
# u_SS = seqTxb(symTriSS, T, u0, seqlength)
u_SSexp = seqTxb(symTriSSexp, T, u0, seqlength)

# display(norm(u_UtU - u_SS))
display(norm(u_UtU - u_SSexp))
# display(norm(u_SSexp - u_SS))

1.1641162945323024e-14

In [45]:
@btime u_UtU = seqTxb(CholeskyUtU, T, u0, seqlength);

  170.541 μs (308 allocations: 496.20 KiB)


In [46]:
@btime u_SS = seqTxb(symTriSS, T, u0, seqlength);

  762.583 μs (3971 allocations: 3.36 MiB)


In [47]:
@btime u_SSexp = seqTxb(symTriSSexp, T, u0, seqlength);

  240.466 ms (887214 allocations: 47.29 MiB)


In [34]:
"a"+"a"

MethodError: MethodError: no method matching +(::String, ::String)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655

In [36]:
a = "a"
append!(a, "dsds")

MethodError: MethodError: no method matching append!(::String, ::String)
Closest candidates are:
  append!(!Matched::BitVector, ::Any) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/bitarray.jl:782
  append!(!Matched::AbstractVector, ::Any) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/array.jl:1050
  append!(!Matched::AbstractVector, ::Any...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/array.jl:1053

In [1]:
solve(rand(3,3), rand(3))

UndefVarError: UndefVarError: solve not defined

In [2]:
rand(3)

3-element Vector{Float64}:
 0.876367452388326
 0.3202043172263417
 0.5742610242200911